<a href="https://colab.research.google.com/github/hamin-shim/Prometheus_Cooker/blob/Embedding_model/Embedding(upstage_api).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [10]:

# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import torch
from openai import OpenAI
import torch.nn.functional as F

# OpenAI 클라이언트 설정
client = OpenAI(
    api_key="사용자_API",  # 실제 API 키로 교체해야 합니다
    base_url="https://api.upstage.ai/v1/solar"
)

# 데이터 로딩
df = pd.read_excel('./sampled.xlsx')

# 결측값 처리
df['DATA'] = df['DATA'].fillna('')

# label이 1인 데이터 필터링
filtered_df = df[df['TARGET'] == 1]

# 문장과 메뉴 이름을 리스트로 변환
texts = filtered_df['DATA'].tolist()
menu_names = filtered_df['MENU_NAME'].tolist()  # MENU_NAME 컬럼 추가

# OpenAI 임베딩 생성 함수
def get_embeddings(texts, batch_size=100):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        response = client.embeddings.create(
            input=batch_texts,
            model="solar-embedding-1-large-query"
        )
        batch_embeddings = [np.array(embedding.embedding) for embedding in response.data]
        embeddings.extend(batch_embeddings)
    return embeddings

# 필터링된 데이터의 임베딩 생성
embeddings = get_embeddings(texts)

# 사용자 입력 문장
queries = ["달콤하면서도 약간의 신맛이 조화를 이루어, 마치 여름 과일을 한입 베어 문 듯한 상쾌함을 느낄 수 있는 음식 추천해줘"]

# 코사인 유사도를 기준으로 입력 문장과 가장 유사한 5개 음식 추출
top_k = 5
for query in queries:
    query_embedding = get_embeddings([query])[0]
    query_embedding = torch.tensor(query_embedding)

    # 코사인 유사도 계산
    embeddings_tensor = torch.tensor(embeddings)
    cos_scores = F.cosine_similarity(query_embedding.unsqueeze(0), embeddings_tensor, dim=1)

    # 상위 top_k 결과 추출
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("사용자 입력 :", query)
    print("\nTop 5 음식 추천:")

    for idx in top_results.indices:
        print(menu_names[idx].strip()," : ",texts[idx].strip(),  "(Score: %.4f)" % (cos_scores[idx].item()))






사용자 입력 : 달콤하면서도 약간의 신맛이 조화를 이루어, 마치 여름 과일을 한입 베어 문 듯한 상쾌함을 느낄 수 있는 음식 추천해줘

Top 5 음식 추천:
토마토샐러드  :  떠먹는 토마토 샐러드는 토마토의 신선함과 소스의 달콤함이 너무 맛있답니다 (Score: 0.5816)
사과양배추샐러드  :  유튜브에서 보고 따라해봤어요.  상큼한게 맛있어요. (Score: 0.5641)
뚝배기배추국  :  얼큰하고 시원~한 맛이에요 (Score: 0.5440)
복숭아멘보샤  :  달콤한 데일리 복숭아가 탱글탱글 새우를 만나 튀김으로 하나가 되었다! 달콤 바삭한 복숭아 멘보샤  자칫하면 느끼할 수 있는 튀김요리의 신의 한 수  '새콤달콤 복숭아칠리소스와 팬에 구워 달콤함이 배가된 복숭아'를 곁들여 먹는 맛난 간식! 한입 크기로 파티 핑거푸드로도 좋아요~ (Score: 0.5418)
찜닭  :  달달한 간장양념이 가미된 찜닭요리를 해 봤어요! :) 여름보양식으로 딱이예요. (Score: 0.5380)
